In [ ]:
# 🧠 Predictive Maintenance – Test načítání dat s filtrem
# 📦 Importy:

import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(".."))
from data.database_connection import get_engine
from data.data_loader import (
    get_unique_equipment_ids,
    get_unique_kpi_ids,
    get_unique_object_ids,
    fetch_sensor_data_filtered
)

In [ ]:
# --- 1. Získání dostupných hodnot ---

print("🔎 Načítám dostupné hodnoty z databáze...")

equipment_ids = get_unique_equipment_ids()
kpi_ids = get_unique_kpi_ids()
object_ids = get_unique_object_ids()

print(f"✅ {len(equipment_ids)} unikátních equipment_id")
print(f"✅ {len(kpi_ids)} unikátních kpi_id")
print(f"✅ {len(object_ids)} unikátních object_id")

In [ ]:
# --- 2. Ruční výběr hodnot pro testování ---

# ZDE MĚŇ PARAMETRY PRO TEST
selected_object_id = 1
selected_kpi_id = None # nebo konkrétní KPI stylem [1,2,3] 
start_time = "2025-01-01 00:00:00"
end_time = "2025-03-01 00:00:00"

print(f"\n🎯 Vybrané parametry:\n- FC OBJECT ID: {selected_object_id}\n- KPI ID: {selected_kpi_id}\n- Období: {start_time} – {end_time}")

In [ ]:
# --- 3. Filtrování dat ---
filtered_df = fetch_sensor_data_filtered(
    object_id=selected_object_id,
    kpi_ids=selected_kpi_id,
    start_time=start_time,
    end_time=end_time
)

In [ ]:
# --- 4. Náhled výsledku ---
print(f"\n📊 Načteno {len(filtered_df)} řádků.")
filtered_df.head(3)

In [ ]:
# Předzpracování vyžaduje sloupce: timestamp, object_id, kpi_id, value

df = filtered_df.rename(columns={
    'data_timestamp': 'timestamp',
    'id_fc_object': 'object_id',
    'id_fc_kpi_definition': 'kpi_id',
    'value': 'value'
})

In [ ]:
# Spuštění předzpracování
from utils.preprocessing import preprocess_sensor_data
from utils.preprocessing2 import preprocess_sensor_data2

processed = preprocess_sensor_data2(df, impute=True)

In [ ]:
# Vezmeme první stroj (object_id) a zobrazíme jeho dataframe
if processed:
    first_object_id = list(processed.keys())[0]
    print(f"🔍 Výstup pro object_id: {first_object_id}")
    display(processed[first_object_id].head(5))
else:
    print("⚠️ Nebyla nalezena žádná data po zpracování.")

In [ ]:
from utils.visualization import (
    plot_kpi_raw_trends,
    plot_kpi_rolling,
    plot_correlation_heatmap,
    plot_feature_distributions,
    plot_missing_data_pattern
)

In [ ]:
# Trendy pro KPI
plot_kpi_raw_trends(
    df=processed,            # Vstupní data – buď slovník {object_id: DataFrame}, nebo přímo DataFrame
    object_id=1,             # ID objektu – použije se pouze pokud je df slovník

    kpi_ids=None,            # Seznam KPI, které chceš vykreslit (např. [7, 8, 23])
                             # Pokud None, zobrazí se všechny číselné sloupce (raw i engineered)

    feature_types=[          # Volitelné zúžení podle typu feature:
        'raw'              
        #   'raw' Originální KPI hodnoty (např. '7', '8', '23')             
        #   'mean' Rolling průměry (např. 'mean_6')               
        #   'std' Rolling směrodatné odchylky (např. 'std_3')       
        #   'diff1' První diference
        #   'diff2' Druhá diference        
        #   'pct_change' Procentuální změna        
        #   'time_since' Speciální sloupec 'time_since_last'
        #   'rolling' Zástupný typ – zahrne mean + std dohromady
    ],

    agg_freq='1h',           # Agregační frekvence (resampling), např. '1min', '15t', '1h'
                             # Použije se na časovou osu – např. 1h = průměrování po hodinách

    highlight_missing=True,   # Zvýraznit chybějící hodnoty červenými značkami (např. kde je NaN)
    start_time=None,   # Počáteční časový filtr buďto nechat "start_time" nebo konkrétní čas (např. "2025-01-01 00:00:00")
    end_time=None       # Koncový časový filtr buďto nechat "end_time" nebo konkrétní čas (např. "2025-03-01 00:00:00")
)


In [ ]:
# Rolling mean s oknem 6 vzorků
plot_kpi_rolling(
    df=processed,                # Dict nebo DataFrame
    object_id=1,                 # ID objektu (stroje)
    kpi_ids=[1],                # Seznam KPI (např. [23, 24]) nebo None = všechny KPI
    window=6,                    # Rolling okno (počet časových kroků), např. 6
    method='mean',                # Typ výpočtu: 'mean', 'std', 'max', 'min', ...
    start_time=start_time,       # Počáteční časový filtr
    end_time=end_time            # Koncový časový filtr
)

In [ ]:
plot_correlation_heatmap(
    df=processed,            # Vstupní data – buď DataFrame nebo dict ve formátu {object_id: DataFrame}
    object_id=1,             # ID objektu – použije se jako klíč pro výběr ze slovníku (pokud je df typu dict)

    kpi_ids=None,            # Volitelný výběr konkrétních KPI podle ID – např. [7, 8, 23]
                             # Pokud None, použijí se všechny KPI (dle výběru feature types níže)

    feature_types=[          # Výběr typů sloupců (feature), které se mají zahrnout do korelační matice:
        'raw'           
        #   'raw' Pouze původní KPI sloupce, např. '7', '8', '23'
        #   'mean' Rolling průměr, např. '7_mean_6'
        #   'std' Rolling směrodatná odchylka, např. '7_std_6'
        #   'diff1' První diference, např. '7_diff1'
        #   'diff2' Druhá diference, např. '7_diff2'
        #   'pct_change' Procentuální změna, např. '7_pct_change'
        #   'time_since' Speciální feature např. 'time_since_last'
        #   'rolling' Zahrne všechny rolling features (mean_, std_), alias pro pohodlí
    ],
    start_time=start_time,       # Počáteční časový filtr
    end_time=end_time            # Koncový časový filtr
)

In [ ]:
# Histogramy dat
plot_feature_distributions(
    df=processed,              # Dict nebo DataFrame
    object_id=1,               # ID objektu
    kpi_ids=[1],              # Konkrétní KPI (např. [8, 23, 25]); None = všechny
    raw=False,                 # True = použít neskalovaná data (pouze originální KPI)
    log=False,                  # True = logaritmická osa pro histogram
    start_time=start_time,       # Počáteční časový filtr
    end_time=end_time            # Koncový časový filtr
)

In [ ]:
# Vizualizace chybějících dat
plot_missing_data_pattern(
    df=processed,              # Dict nebo DataFrame
    object_id=1,               # ID objektu
    n_rows=500,                 # Počet časových řádků k zobrazení (např. 500 = prvních 500 timestamps)
    start_time=start_time,       # Počáteční časový filtr
    end_time=end_time            # Koncový časový filtr
)